In [ ]:
import cv2
import pyautogui
import mss
import numpy as np

pyautogui.FAILSAFE = True

In [ ]:
# Setup
threshold = 0.2  # Acceptable detection error
min_th, max_th = (1-threshold, 1+threshold)
img_path = 'mahjong.png'
pattern_path = 'pattern.png'

In [ ]:
def get_ratios():
    """Helper function to understand a tile shape.
    Unfortunately, resizing game window can change tile shape,
    thus different approach have to be considered."""
    
    img_rgb = cv2.imread(img_path, 0)
    i_w, i_h = img_rgb.shape[::-1]
    template = cv2.imread(pattern_path, 0)
    t_w, t_h = template.shape[::-1]
    rec_ratio = t_w/t_h  # Expected tile ratio
    height_ratio = t_h/i_h  # Expected tile-to-board height ratio
    return rec_ratio, height_ratio, i_w, i_h

rec_ratio, height_ratio, i_w, i_h = get_ratios()

In [ ]:
def grab_screen():
    """Capture a screen image."""
    try:
        img = np.array(sct.grab(monitor))
    except NameError:
        return "Initiate mss first."
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

In [ ]:
def loc_tiles(screen):
    """Localize tiles within an image."""
    
    tiles = []
    img_gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
    binary = cv2.bitwise_not(img_gray)
    (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if (w/h > (rec_ratio * min_th)
            and w/h < (rec_ratio * max_th)
            and h/i_h > (height_ratio * min_th)
            and h/i_h < (height_ratio * max_th)):
            tiles.append((x, y, h, w))
    print('%d tiles detected.' % len(tiles))
    return tiles

In [ ]:
def clickable(tile):
    """Check whether tile is clickable."""
    x, y, w, h = tile

    img = grab_screen()
    pre_click = img[y:y+h, x:x+w]

    pyautogui.click(x=(x+w/2), y=(y+h/2))

    img = grab_screen()
    post_click = img[y:y+h, x:x+w]

    difference = cv2.subtract(pre_click, post_click)
    b, g, r = cv2.split(difference)
    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        return False
    else:
        return tile

In [ ]:
def loc_siblings(tile, clickables=[]):
    """Finds matching tiles."""
    x, y, h, w = tile
    screen = grab_screen()
    look_for = screen[y:y+h, x:x+w]
    result = cv2.matchTemplate(screen, look_for, cv2.TM_CCOEFF_NORMED)
    pattern_threshold = 0.8
    loc = np.where(result >= pattern_threshold) #  (array(y), array(x))
    sibling = zip(*loc[::-1])
    last_pressed = [0, 0]  # Way to limit double-taping tiles
    
    if loc[::-1][0].size == 1:
        # Found only 1 tile of this type.
        return (*list(sibling)[0],h,w)
    
    target = []
    for pt in sibling:
        pt_x = pt[0] + w/2  # Center of tile
        pt_y = pt[1] + h/2  #

        if abs(last_pressed[0] - pt[0]) < 20 and abs(last_pressed[1] - pt[1]) < 20:
            #  Avoid clicking same tile twice.
            pass
        else:
            for ctile in clickables:
                cx, cy, ch, cw = ctile
                if (cx < pt_x < cx+cw
                    and cy < pt_y < cy+ch):
                    clickables.pop(clickables.index(ctile))
                    target.append((pt_x, pt_y))
                    last_pressed = pt
        
    if len(target) > 1: # Click only if clickable pair is detected.
        for pew in target:
            pyautogui.click(*pew)
            # time.sleep(1)

In [ ]:
with mss.mss() as sct:
    # Part of the screen to capture
    monitor = {"top": 0, "left": 0, "width": 960, "height": 1080}

    while "Screen capturing":
        last_time = time.time()

        img = grab_screen()
        tiles = loc_tiles(img)
        tiles_count = len(tiles)
        clickable_tiles = []
        lonely = []
        
        for i, tile in enumerate(tiles):
            clicked = clickable(tile)
            if clicked is not False:
                clickable_tiles.append(tile)
        
        clickable(clickable_tiles[-1]) # Unclick last clickable tile.
        
        img = grab_screen()
        check_tiles = loc_tiles(img)
        if len(check_tiles) == len(tiles):
            print('Found {} clickable tiles.'.format(len(clickable_tiles)))
            for tile in clickable_tiles:
                output = loc_siblings(tile, clickable_tiles)
        else:
            pass
            
        print("fps: {}".format(1 / (time.time() - last_time)))

        # Press "q" to quit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break